## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/134bert-base-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_1.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_2.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_3.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_4.h5
/kaggle/input/134bert-base-best/134-BERT_base_unc raw_model_fold_5.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.939826,0.616974,0.280316,0.497946,0.537702,0.497758,0.653974,0.642878,0.559083,...,0.883290,0.895463,0.574437,0.952517,0.957001,0.796382,0.071264,0.068385,0.809621,0.911184
1,46,0.887289,0.526726,0.007461,0.732969,0.769595,0.905026,0.554527,0.475386,0.075901,...,0.735075,0.947682,0.675055,0.968700,0.979636,0.876880,0.866095,0.124664,0.121727,0.884658
2,70,0.926241,0.676852,0.049381,0.762637,0.803043,0.888671,0.590819,0.491223,0.151679,...,0.856008,0.939226,0.663460,0.969532,0.976108,0.866046,0.094467,0.063877,0.861809,0.926636
3,132,0.885368,0.427033,0.009424,0.701466,0.764280,0.903889,0.549289,0.431740,0.133754,...,0.706547,0.951945,0.670305,0.968521,0.983972,0.890746,0.773680,0.155213,0.566496,0.916443
4,200,0.927322,0.466872,0.072437,0.787938,0.687041,0.761965,0.626558,0.569240,0.104227,...,0.724737,0.913786,0.645507,0.965288,0.968915,0.847103,0.112828,0.096978,0.711812,0.916913


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.898452,0.596257,0.042060,0.708319,0.784484,0.831307,0.573944,0.487297,0.233198,...,0.796163,0.935855,0.666834,0.963728,0.975013,0.868359,0.464355,0.135324,0.565286,0.912400
std,2812.670060,0.037807,0.120842,0.064279,0.091472,0.100356,0.131057,0.044951,0.081058,0.176242,...,0.074201,0.019130,0.040562,0.009287,0.008413,0.031396,0.298578,0.057310,0.252582,0.017187
min,39.000000,0.788124,0.370670,0.003742,0.312584,0.374407,0.274160,0.476260,0.350206,0.013645,...,0.596922,0.836859,0.548675,0.925096,0.928203,0.742259,0.010611,0.014202,0.048970,0.854705
25%,2572.000000,0.874486,0.484985,0.009714,0.658483,0.745185,0.802814,0.540479,0.425124,0.097583,...,0.733063,0.925153,0.639660,0.958243,0.970997,0.847075,0.147543,0.093265,0.357300,0.900843
50%,5093.000000,0.900781,0.587215,0.015938,0.708264,0.798082,0.883662,0.569662,0.467364,0.162111,...,0.802971,0.938743,0.665335,0.964887,0.976288,0.870285,0.515247,0.136308,0.570807,0.914228
75%,7482.000000,0.927396,0.691965,0.040960,0.765994,0.849237,0.914264,0.602660,0.533539,0.336760,...,0.861834,0.949331,0.693138,0.970517,0.981084,0.890936,0.742405,0.174814,0.776885,0.925124
max,9640.000000,0.971536,0.891698,0.526309,0.940783,0.964631,0.958155,0.688013,0.728125,0.731338,...,0.949800,0.973562,0.793428,0.982680,0.990625,0.943170,0.920119,0.293889,0.984132,0.949648
